# **Preliminaries:** Install and import modules

In [62]:
#@title [RUN] install
!pip install networkx
!pip install mycolorpy
!pip install colorama
!pip install ogb

import torch
import os
!pip install torch-geometric torch-scatter torch-sparse torch-cluster -f https://data.pyg.org/whl/torch-{torch.__version__}.html


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://data.pyg.org/whl/torch-1.13.1+cu116.html


In [63]:
#@title [RUN] Import modules
import numpy as np
import seaborn as sns
import math
import itertools
import scipy as sp
import random

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch_geometric
from torch_geometric.datasets import Planetoid, Coauthor
from torch_scatter import scatter_mean, scatter_max, scatter_sum
from torch_geometric.utils import to_dense_adj
from torch.nn import Embedding
from torch_geometric.typing import Adj
from ogb.nodeproppred import PygNodePropPredDataset
from torch_geometric.loader import NeighborLoader
from torch_geometric.utils import to_scipy_sparse_matrix, degree

#For FastRP
from scipy.sparse import coo_matrix, csr_matrix, csc_matrix, spdiags
from sklearn.preprocessing import normalize, scale, MultiLabelBinarizer
from sklearn import random_projection


import pdb
from datetime import datetime

#for nice visualisations
import networkx as nx
import matplotlib.pyplot as plt

from mycolorpy import colorlist as mcp
import matplotlib.cm as cm

from typing import Mapping, Tuple, Sequence, List
import colorama

import scipy.linalg
from scipy.linalg import block_diag

In [64]:
####### PLOTS #######

def update_stats(training_stats, epoch_stats):
    """ Store metrics along the training
    Args:
      epoch_stats: dict containg metrics about one epoch
      training_stats: dict containing lists of metrics along training
    Returns:
      updated training_stats
    """
    if training_stats is None:
        training_stats = {}
        for key in epoch_stats.keys():
            training_stats[key] = []
    for key,val in epoch_stats.items():
        training_stats[key].append(val)
    return training_stats

def plot_stats(training_stats, figsize=(5, 5), name=""):
    """ Create one plot for each metric stored in training_stats
    """
    stats_names = [key[6:] for key in training_stats.keys() if key.startswith('train_')]
    f, ax = plt.subplots(len(stats_names), 1, figsize=figsize)
    if len(stats_names)==1:
        ax = np.array([ax])
    for key, axx in zip(stats_names, ax.reshape(-1,)):
        axx.plot(
            training_stats['epoch'],
            training_stats[f'train_{key}'],
            label=f"Training {key}")
        axx.plot(
            training_stats['epoch'],
            training_stats[f'val_{key}'],
            label=f"Validation {key}")
        axx.set_xlabel("Training epoch")
        axx.set_ylabel(key)
        axx.legend()
    plt.title(name)


def get_color_coded_str(i, color):
    return "\033[3{}m{}\033[0m".format(int(color), int(i))

def print_color_numpy(map, list_graphs):
    """ print matrix map in color according to list_graphs
    """
    list_blocks = []
    for i,graph in enumerate(list_graphs):
        block_i = (i+1)*np.ones((graph.num_nodes,graph.num_nodes))
        list_blocks += [block_i]
    block_color = block_diag(*list_blocks)
    
    map_modified = np.vectorize(get_color_coded_str)(map, block_color)
    print("\n".join([" ".join(["{}"]*map.shape[0])]*map.shape[1]).format(*[x for y in map_modified.tolist() for x in y]))

# Cora dataset



In [65]:
cora_dataset = Planetoid("/tmp/cora", name="cora", split="full")
cora_data = cora_dataset[0]
cora_data

Data(x=[2708, 1433], edge_index=[2, 10556], y=[2708], train_mask=[2708], val_mask=[2708], test_mask=[2708])

In [ ]:
print("Training class sizes")
print(torch.bincount(cora_dataset[0].y[cora_dataset[0].train_mask]))
print("Validation class sizes")
print(torch.bincount(cora_dataset[0].y[cora_dataset[0].val_mask]))
print("Test class sizes")
print(torch.bincount(cora_dataset[0].y[cora_dataset[0].test_mask]))

# OBGN-ARVIX dataset

In [66]:
d_name = "ogbn-arxiv"

dataset = PygNodePropPredDataset(name = d_name)

split_idx = dataset.get_idx_split()
train_idx, valid_idx, test_idx = split_idx["train"], split_idx["valid"], split_idx["test"]
arxiv_data = dataset[0]
arxiv_data.y = arxiv_data.y.squeeze()
arxiv_data.node_year = arxiv_data.node_year.squeeze()
arxiv_data

Downloaded 0.08 GB: 100%|██████████| 81/81 [00:09<00:00,  8.81it/s]


Extracting dataset/arxiv.zip


Processing...


Loading necessary files...
This might take a while.
Processing graphs...


100%|██████████| 1/1 [00:00<00:00, 1988.76it/s]


Converting graphs into PyG objects...


100%|██████████| 1/1 [00:00<00:00, 4917.12it/s]

Saving...



Done!


Data(num_nodes=169343, edge_index=[2, 1166243], x=[169343, 128], node_year=[169343], y=[169343])

In [ ]:
print("Training class sizes")
print(torch.bincount(arxiv_data.y[train_idx]))
print("Validation class sizes")
print(torch.bincount(arxiv_data.y[valid_idx]))
print("Test class sizes")
print(torch.bincount(arxiv_data.y[test_idx]))

#Coauthor dataset

In [67]:
cs_dataset = Coauthor("/tmp/coauthor", name="CS")
cs_data = cs_dataset[0]
cs_data

Processing...
Done!


Data(x=[18333, 6805], edge_index=[2, 163788], y=[18333])

In [68]:
# Create manual split, do 60:20:20 across classes
num_classes_cs = 15
train_mask_cs_indices = []
val_mask_cs_indices = []
test_mask_cs_indices = []
cs_labels = cs_data.y
for i in range(num_classes_cs):

  class_i = np.where(cs_labels == i)[0]
  np.random.seed(0)
  np.random.shuffle(class_i)

  num_samples = len(class_i)
  train_mask_cs_indices += (class_i[:int(num_samples*0.6)]).tolist() 
  val_mask_cs_indices += (class_i[int(num_samples*0.6):int(num_samples*0.8)]).tolist() 
  test_mask_cs_indices += (class_i[int(num_samples*0.8):]).tolist() 

print(len(train_mask_cs_indices), len(val_mask_cs_indices), len(test_mask_cs_indices))
# Create the masks for training
# Test mask 
train_mask_cs = torch.full((len(cs_labels),), False)
train_mask_cs[train_mask_cs_indices] = True
# Val mask
val_mask_cs = torch.full((len(cs_labels),), False)
val_mask_cs[val_mask_cs_indices] = True
# Train mask
test_mask_cs = torch.full((len(cs_labels),), False)
test_mask_cs[test_mask_cs_indices] = True

10993 3668 3672


In [ ]:
print("Training class sizes")
print(torch.bincount(cs_data.y[train_mask_cs]))
print("Validation class sizes")
print(torch.bincount(cs_data.y[val_mask_cs]))
print("Test class sizes")
print(torch.bincount(cs_data.y[test_mask_cs]))

# Data saving / loading

In [69]:
# use google drive for saving and loading information
from google.colab import drive
import pickle
import os

drive.mount('/content/drive')
file_path = '/content/drive/MyDrive/L45_project/'
# create folder if it does not exist already
if not os.path.exists(file_path):
  os.mkdir(file_path) 

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [70]:
def save_training_info(training_stats: dict, node_embedding: torch.Tensor, filename: str):
  # write training data info to a file
  with open(file_path + filename + ".pkl", 'wb') as fp:
    pickle.dump(training_stats, fp)
    print('Training stats saved successfully to file: ' + filename)
  # write node embedding to a file
  torch.save(node_embedding, file_path + filename + "_emb.pt")
  print('Node embedding saved successfully to file: ' + filename)


def load_training_info(filename: str):
  # load training stats dictionary 
  with open(file_path + filename + ".pkl", 'rb') as fp:
    train_stats = pickle.load(fp)
    print('Training stats successfully loaded from file: ' + filename)
  # load node embedding
  node_embedding = torch.load(file_path + filename + "_emb.pt")
  print('Node embedding successfully loaded from file: ' + filename)
  return train_stats, node_embedding

# Final results is a list [seed, test result, [test per class accuracy], [training per class accuracy], [val per class accuracy]]
def save_final_results(final_results: List, filename: str):
  # write training data info to a file
  with open(file_path + filename + ".pkl", 'ab') as fp:
    pickle.dump(final_results, fp)
    print('Final results saved successfully to file: ' + filename)

# Returns an iterator which contains all the results from our various runs
def load_final_results(filename: str):
  with open(file_path + filename + ".pkl", 'rb') as fp:
    print('Final results found in file: ' + filename)
    while True:
      try:
        # This notation creates a generator, which we can then iterate through
        yield pickle.load(fp)
      except EOFError:
        break


In [ ]:
test_dict = {'c':[1,2,3], 'b':[4,5,6]}
test_tensor = torch.tensor([[1., -1.], [1., -1.]])
save_training_info(test_dict, test_tensor, "testing")
recovered_val1, recovered_val2 = load_training_info("testing")
print(recovered_val1, recovered_val2)

Training stats saved successfully to file: testing
Node embedding saved successfully to file: testing
Training stats successfully loaded from file: testing
Node embedding successfully loaded from file: testing
{'c': [1, 2, 3], 'b': [4, 5, 6]} tensor([[ 1., -1.],
        [ 1., -1.]])


# Model Wrappers

In [ ]:
from torch_geometric.nn import GCN

class GCNModelWrapper(GCN):

  def __init__(self, in_channels: int, hidden_channels: int, num_layers: int, out_channels: int):
    # use one less layer as our final graph layer can downsize for us
    # super().__init__(in_channels, hidden_channels, num_layers-1)
    super().__init__(in_channels, hidden_channels, num_layers)
    self.out_channels = out_channels
    self.final_layer = nn.Linear(hidden_channels, out_channels)

  def forward(self, x: torch.Tensor, edge_index: Adj):
    x = super().forward(x, edge_index)
    output = self.final_layer(x)
    return output


In [ ]:
from torch_geometric.nn import GAT

class GATModelWrapper(GAT):

  def __init__(self, in_channels: int, hidden_channels: int, num_layers: int, out_channels: int, v2: bool):
    # Create the model to extract the node embeddings then pass these through a linear layer for classification
    super().__init__(in_channels, hidden_channels, num_layers, v2=v2)
    self.out_channels = out_channels
    self.final_layer = nn.Linear(hidden_channels, out_channels)

  def forward(self, x: torch.Tensor, edge_index: Adj):
    x = super().forward(x, edge_index)
    output = self.final_layer(x)
    return output, x

In [ ]:
from torch_geometric.nn import GraphSAGE

class GraphSAGEModelWrapper(GraphSAGE):

  def __init__(self, in_channels: int, hidden_channels: int, num_layers: int, out_channels: int):
    # Create the model to extract the node embeddings then pass these through a linear layer for classification
    super().__init__(in_channels, hidden_channels, num_layers)
    self.out_channels = out_channels
    self.final_layer = nn.Linear(hidden_channels, out_channels)

  def forward(self, x: torch.Tensor, edge_index: Adj):
    x = super().forward(x, edge_index)
    output = self.final_layer(x)
    return output, x

In [ ]:
from torch_geometric.nn import Node2Vec
from torch import Tensor

class Node2VecWrapper(Node2Vec):
  def __init__(self, edge_index, embedding_size, walk_length, context_size, walks_per_node, num_negative_samples, p, q, sparse, out_channels):
    super().__init__(edge_index, embedding_dim=embedding_size, walk_length=walk_length,
                     context_size=context_size, walks_per_node=walks_per_node,
                     num_negative_samples=num_negative_samples, p=p, q=q, sparse=sparse)
    self.final_layer = nn.Linear(embedding_size, out_channels)
  def forward(self):
    x = super().forward()
    output = F.softmax(self.final_layer(x), dim=1)
    return output, x
  def test(
    self,
    train_z: Tensor,
    train_y: Tensor,
    test_z: Tensor,
    test_y: Tensor,
    solver: str = 'lbfgs',
    multi_class: str = 'auto',
    *args,
    **kwargs,
    ) -> float:
    r"""Evaluates latent space quality via a logistic regression downstream
    task."""
    from sklearn.linear_model import LogisticRegression

    clf = LogisticRegression(solver=solver, multi_class=multi_class, *args,
                            **kwargs).fit(train_z.detach().cpu().numpy(),
                                          train_y.detach().cpu().numpy())
    y_pred = clf.predict(test_z.detach().cpu().numpy())
    return y_pred

In [ ]:
from torch_geometric.nn import GIN

class GINWrapper(GIN):

  def __init__(self, in_channels: int, hidden_channels: int, num_layers: int, out_channels: int):
    # Create the model to extract the node embeddings then pass these through a linear layer for classification
    super().__init__(in_channels, hidden_channels, num_layers)
    self.out_channels = out_channels
    self.final_layer = nn.Linear(hidden_channels, out_channels)

  def forward(self, x: torch.Tensor, edge_index: Adj):
    x = super().forward(x, edge_index)
    output = self.final_layer(x)
    return output, x

# Training code



In [71]:
# @title [RUN] Hyperparameters GNN

NUM_EPOCHS_CORA =  10 #@param {type:"integer"}
NUM_EPOCHS_ARVIX =  110 #@param {type:"integer"}
LR         = 0.01 #@param {type:"number"}
HIDDEN_DIM = 128  #@param {type:"integer"}


In [ ]:
# Code taken from L45 practical notebook
def train_gnn(X, edge_indices, y, mask, model, optimiser, device):
    model.train()
    # Put data on device
    X = X.to(device)
    edge_indices = edge_indices.to(device)
    y = y.to(device)
    mask = mask.to(device)
    # Train
    optimiser.zero_grad()
    y_out, _ = model(X, edge_indices)
    y_hat = y_out[mask]
    loss = F.cross_entropy(y_hat, y)
    loss.backward()
    optimiser.step()
    return loss.data

# Training loop using subgraph batching from paper 'Inductive Representation Learning on Large Graphs' https://arxiv.org/pdf/1706.02216.pdf
def train_gnn_subgraph(data_batch, model, optimiser, device):
  total_loss = 0
  for batch in data_batch:
    # Put batch in device
    batch = batch.to(device)
    # Do training loop
    batch_size = batch.batch_size
    optimiser.zero_grad()
    y_out, _ = model(batch.x, batch.edge_index)
    y_out = y_out[:batch_size]
    batch_y = batch.y[:batch_size]
    batch_y = torch.reshape(batch_y, (-1,))
    loss = F.cross_entropy(y_out, batch_y)
    loss.backward()
    optimiser.step()
    # Keep a running total of the loss
    total_loss += float(loss)

  # Get the average loss across all the batches
  loss = total_loss / len(data_batch)
  return loss

def evaluate_gnn(X, edge_indices, y, mask, model, num_classes, device):
    model.eval()
    # Put data on device
    X = X.to(device)
    edge_indices = edge_indices.to(device)
    y = y.to(device)
    mask = mask.to(device)
    # Evaluate
    with torch.no_grad():
      y_out, node_embeddings = model(X, edge_indices)
    y_hat = y_out[mask]
    y_hat = y_hat.data.max(1)[1]
    num_correct = y_hat.eq(y.data).sum()
    num_total = len(y)
    accuracy = 100.0 * (num_correct/num_total)

    # calculate per class accuracy
    values, counts = torch.unique(y_hat[y_hat == y.data], return_counts=True)
    per_class_counts = torch.zeros(num_classes)
    # make sure per_class_counts is on the correct device
    per_class_counts = per_class_counts.to(device)
    # allocate the number of counts per class
    for i, x in enumerate(values):
      per_class_counts[x] = counts[i]
    # find total number of data points per class in the split
    total_per_class = torch.bincount(y.data)
    per_class_accuracy = torch.div(per_class_counts, total_per_class)

    return accuracy, per_class_accuracy, node_embeddings
    
# Training loop
def train_eval_loop_gnn(model, edge_indices, train_x, train_y, train_mask, valid_x, valid_y, valid_mask, 
                             test_x, test_y, test_mask, num_classes, seed, filename, device, Cora, subgraph_batches=None):
    optimiser = optim.Adam(model.parameters(), lr=LR)
    training_stats = None
    # Choose number of epochs
    NUM_EPOCHS = NUM_EPOCHS_CORA if Cora else NUM_EPOCHS_ARVIX
    # Training loop
    for epoch in range(NUM_EPOCHS):
        # If subgraph batching is not provided, use the full graph for training. Otherwise use subgraph batch training regime
        if subgraph_batches is None:
          train_loss = train_gnn(train_x, edge_indices, train_y, train_mask, model, optimiser, device)
        else:
          train_loss = train_gnn_subgraph(subgraph_batches, model, optimiser, device)
        # Calculate accuracy on full graph  
        train_acc, train_class_acc, _ = evaluate_gnn(train_x, edge_indices, train_y, train_mask, model, num_classes, device)
        valid_acc, valid_class_acc, _ = evaluate_gnn(valid_x, edge_indices, valid_y, valid_mask, model, num_classes, device)
        if epoch % 10 == 0 or epoch == (NUM_EPOCHS-1):
            print(f"Epoch {epoch} with train loss: {train_loss:.3f} train accuracy: {train_acc:.3f} validation accuracy: {valid_acc:.3f}")
            print("Per class train accuracy: ", train_class_acc)
            print("Per class val accuracy: ", valid_class_acc)
        # store the loss and the accuracy for the final plot
        epoch_stats = {'train_acc': train_acc, 'val_acc': valid_acc, 'epoch':epoch}
        training_stats = update_stats(training_stats, epoch_stats)

    # Lets look at our final test performance
    # Only need to get the node embeddings once, take from the training evaluation call
    test_acc, test_class_acc, node_embeddings = evaluate_gnn(test_x, edge_indices, test_y, test_mask, model, num_classes, device)
    print(f"Our final test accuracy for the GNN is: {test_acc:.3f}")
    print("Final per class accuracy on test set: ", test_class_acc)

    # Save training stats if on final iteration of the run
    save_training_info(training_stats, node_embeddings, filename+"_"+str(seed))
    # Save final results
    final_results_list = [seed, test_acc, test_class_acc, train_class_acc, valid_class_acc]
    save_final_results(final_results_list, filename)
    # Save final model weights incase we want to do further inference later
    torch.save(model.state_dict(), file_path+filename+"_" + str(seed) + "_model.pt")
    return training_stats

In [72]:
def set_seeds(seed):
  print("SETTING SEEDS TO: ", str(seed))
  # seed the potential sources of randomness
  torch.manual_seed(seed)
  np.random.seed(seed)
  random.seed(seed)

In [78]:
# CHANGE: To name of model being tested
filename = "FastRP-coauthor"
dataset = "Coauthor"
# use 30 seeds which have been randomly generated using seed_list = [np.random.randint(4294967296 - 1) for i in range(30)]
seeds = [4193977854, 1863727779, 170173784, 2342954646, 116846604, 2105922959, 2739899259, 1024258131, 806299656, 880019963, 1818027900, 2135956485, 3710910636, 1517964140, 4083009686, 2455059856, 400225693, 89475662, 361232447, 3647665043, 1221215631, 2036056847, 1860537279, 516507873, 3692371949, 3300171104, 2794978777, 3303475786, 2952735006, 572297925]

# create folder for saving all model info into if it does not exist already
if not os.path.exists(file_path+filename+"/"):
  os.mkdir(file_path+filename+"/")

if dataset == "Cora":
  print("Using Cora dataset")
  # Get the edge indices and node features for our model. General set up variables for running with all the models
  edge_indices = cora_data.edge_index
  node_features = cora_data.x
  neighbour_dataset = cora_data

  # Get masks and training labels for each split
  train_mask = cora_data.train_mask
  train_y = cora_data.y[train_mask]
  valid_mask = cora_data.val_mask
  valid_y = cora_data.y[valid_mask]
  test_mask = cora_data.test_mask
  test_y = cora_data.y[test_mask]

  num_classes = 7
  is_cora=True

elif dataset=="Coauthor":
  print("Using Coauthor dataset")
  # Get the edge indices and node features for our model. General set up variables for running with all the models
  edge_indices = cs_data.edge_index
  node_features = cs_data.x
  neighbour_dataset = cs_data

  # Get masks and training labels for each split
  train_mask = train_mask_cs
  train_y = cs_data.y[train_mask]
  valid_mask = val_mask_cs
  valid_y = cs_data.y[valid_mask]
  test_mask = test_mask_cs
  test_y = cs_data.y[test_mask]

  num_classes = 15
  is_cora=True

# Otherwise we are using arvix dataset
else:
  print("Using Arvix dataset")
  # Get the edge indices and node features for our model
  edge_indices = arxiv_data.edge_index
  node_features = arxiv_data.x
  neighbour_dataset = arxiv_data

  # Get masks and training labels for each split
  train_mask = train_idx
  train_y = arxiv_data.y[train_mask]
  valid_mask = valid_idx
  valid_y = arxiv_data.y[valid_mask]
  test_mask = test_idx
  test_y = arxiv_data.y[test_mask]

  num_classes = 40
  is_cora = False


Using Coauthor dataset


# Training Loops

In [ ]:
# Use to flush GPU memory if it gets too full
import gc
torch.cuda.empty_cache()
gc.collect()

In [ ]:
# General training loop for all models except GraphSAGE, using the whole graph in training instead of using subgraph batching
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
for seed in seeds:
  set_seeds(seed)
  # Create the model
  model = GATModelWrapper(in_channels = node_features.shape[-1], hidden_channels = HIDDEN_DIM, num_layers=1, out_channels=num_classes, v2=True)
  model = model.to(device)

  # Run training loop
  print("TRAINING WITH SEED: ", str(seed))
  train_stats_cora = train_eval_loop_gnn(model, edge_indices, node_features, train_y, train_mask, 
                                            node_features, valid_y, valid_mask, node_features, test_y, test_mask, num_classes, seed, filename+"/"+filename, device, is_cora)
  # Print out graphs if not using GPU
  if device == torch.device('cpu'):
    plot_stats(train_stats_cora, name=filename)

In [ ]:
# Training loop for GraphSAGE which using subgraph batches instead of the entire graph
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
for seed in seeds:
  set_seeds(seed)
  # Original paper uses neighbourhood sizes  S1 = 25 and S2 = 10 so this is what we use
  train_loader = NeighborLoader(neighbour_dataset, num_neighbors = [25, 10], batch_size=1024, input_nodes=train_mask)

  # Create the model
  model = GraphSAGEModelWrapper(in_channels = node_features.shape[-1], hidden_channels = HIDDEN_DIM, num_layers=1, out_channels=num_classes)
  model = model.to(device)

  # Run training loop
  print("TRAINING WITH SEED: ", str(seed))
  train_stats_cora = train_eval_loop_gnn(model, edge_indices, node_features, train_y, train_mask, 
                                            node_features, valid_y, valid_mask, node_features, test_y, test_mask, num_classes, seed, filename+"/"+filename, device, is_cora, subgraph_batches=train_loader)
  # Print out graphs if not using GPU
  if device == torch.device('cpu'):
    plot_stats(train_stats_cora, name=filename)

In [ ]:
# Training loop for GraphSAGE which using subgraph batches instead of the entire graph
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
for seed in seeds:
  set_seeds(seed)
  # Original paper uses neighbourhood sizes  S1 = 25 and S2 = 10 so this is what we use
  train_loader = NeighborLoader(neighbour_dataset, num_neighbors = [25, 10], batch_size=1024, input_nodes=train_mask)

  # Create the model
  model = GINWrapper(in_channels = node_features.shape[-1], hidden_channels = HIDDEN_DIM, num_layers=1, out_channels=num_classes)
  model = model.to(device)

  # Run training loop
  print("TRAINING WITH SEED: ", str(seed))
  train_stats_cora = train_eval_loop_gnn(model, edge_indices, node_features, train_y, train_mask, 
                                            node_features, valid_y, valid_mask, node_features, test_y, test_mask, num_classes, seed, filename+"/"+filename, device, is_cora, subgraph_batches=train_loader)
  # Print out graphs if not using GPU
  if device == torch.device('cpu'):
    plot_stats(train_stats_cora, name=filename)

# TESTING LOADING

In [ ]:
final_results = load_final_results(filename)
for r in final_results:
  print(r)

In [ ]:
training_stats_1, embedding = load_training_info(filename+"_1")
plot_stats(training_stats_1, name="Testing")
print(embedding)
print(node_features)

In [ ]:
# Loading stored model weights
model = GATModelWrapper(in_channels = node_features.shape[-1], hidden_channels = node_features.shape[-1], num_layers=1, out_channels=num_classes, v2=True)
model.load_state_dict(torch.load(file_path+filename+"/"+"GATV2_1_model.pt"))
model.eval()

- Plot graph with average training stats
- Save node embeddings for each run
- Save training stats for each run
- Save test accuracy for each run


# FastRP

In [79]:
class FastRPEmbeddingWrapper(nn.Module):
  def __init__(self, input_dim, num_classes):
      super().__init__()
      self.linear = nn.Linear(input_dim, num_classes)

  def forward(self, x):
      x = self.linear(x)
      return x

In [75]:
# Copied from https://github.com/GTmac/FastRP/blob/master/fastrp.py
# projection method: choose from Gaussian and Sparse
# input matrix: choose from adjacency and transition matrix
# alpha adjusts the weighting of nodes according to their degree
def fastrp_projection(A, seed, q=3, dim=128, projection_method='gaussian', input_matrix='adj', alpha=None):
    assert input_matrix == 'adj' or input_matrix == 'trans'
    assert projection_method == 'gaussian' or projection_method == 'sparse'
    
    N = A.shape[0]
    if input_matrix == 'adj':
        M = A
    else:
        # Change csc_matrix.sum(A) to A.sum as this caused bugs
        normalizer = spdiags(np.squeeze(1.0 / A.sum(axis=1) ), 0, N, N)
        M = normalizer @ A
    # Gaussian projection matrix
    if projection_method == 'gaussian':
        transformer = random_projection.GaussianRandomProjection(n_components=dim, random_state=seed)
    # Sparse projection matrix
    else:
        transformer = random_projection.SparseRandomProjection(n_components=dim, random_state=seed)
    Y = transformer.fit(M)
    # Random projection for A
    if alpha is not None:
      # Change csc_matrix.sum(A) to A.sum as this caused bugs
        Y.components_ = Y.components_ @ spdiags( \
                        np.squeeze(np.power(A.sum(axis=1), alpha)), 0, N, N)
    cur_U = transformer.transform(M)
    U_list = [cur_U]
    
    for i in range(2, q + 1):
        cur_U = M @ cur_U
        U_list.append(cur_U)
    return U_list

# When weights is None, concatenate instead of linearly combines the embeddings from different powers of A
def fastrp_merge(U_list, weights, normalization=False):
    dense_U_list = [_U.todense() for _U in U_list] if type(U_list[0]) == csc_matrix else U_list
    _U_list = [normalize(_U, norm='l2', axis=1) for _U in dense_U_list] if normalization else dense_U_list

    if weights is None:
        return np.concatenate(_U_list, axis=1)
    U = np.zeros_like(_U_list[0])
    for cur_U, weight in zip(_U_list, weights):
        U += cur_U * weight
    # U = scale(U.todense())
    # U = normalize(U.todense(), norm='l2', axis=1)
    return scale(np.asarray(U.todense())) if type(U) == csr_matrix else scale(np.asarray(U))

# A is always the adjacency matrix
# the choice between adj matrix and trans matrix is decided in the conf
def fastrp_wrapper(A, conf, seed):
    U_list = fastrp_projection(A,
                               seed,
                               q=len(conf['weights']),
                               dim=conf['dim'],
                               projection_method=conf['projection_method'],
                               input_matrix=conf['input_matrix'],
                               alpha=conf['alpha'],
    )
    U = fastrp_merge(U_list, conf['weights'], conf['normalization'])
    return U

In [76]:
# Code adpated from L45 practical notebook
def train_embedding_classifier(X, y, mask, model, optimiser, device):
    model.train()
    # Put data on device
    X = X.to(device)
    y = y.to(device)
    mask = mask.to(device)
    # Train
    optimiser.zero_grad()
    y_out = model(X)
    y_hat = y_out[mask]
    loss = F.cross_entropy(y_hat, y)
    loss.backward()
    optimiser.step()
    return loss.data

def evaluate_embedding_classifier(X, y, mask, model, num_classes, device):
    model.eval()
    # Put data on device
    X = X.to(device)
    y = y.to(device)
    mask = mask.to(device)
    # Evaluate
    with torch.no_grad():
      y_out = model(X)
    y_hat = y_out[mask]
    y_hat = y_hat.data.max(1)[1]
    num_correct = y_hat.eq(y.data).sum()
    num_total = len(y)
    accuracy = 100.0 * (num_correct/num_total)

    # calculate per class accuracy
    values, counts = torch.unique(y_hat[y_hat == y.data], return_counts=True)
    per_class_counts = torch.zeros(num_classes)
    # make sure per_class_counts is on the correct device
    per_class_counts = per_class_counts.to(device)
    # allocate the number of counts per class
    for i, x in enumerate(values):
      per_class_counts[x] = counts[i]
    # find total number of data points per class in the split
    total_per_class = torch.bincount(y.data)
    per_class_accuracy = torch.div(per_class_counts, total_per_class)

    return accuracy, per_class_accuracy
    
# Training loop
def train_eval_loop_embedding_classifier(model, embeddings, train_y, train_mask, 
                                         valid_y, valid_mask, test_y, test_mask, num_classes, seed, filename, device, Cora):
    optimiser = optim.Adam(model.parameters(), lr=LR)
    training_stats = None
    # Choose number of epochs
    NUM_EPOCHS = NUM_EPOCHS_CORA if Cora else NUM_EPOCHS_ARVIX
    # Training loop
    for epoch in range(NUM_EPOCHS):
        train_loss = train_embedding_classifier(embeddings, train_y, train_mask, model, optimiser, device)
        # Calculate accuracy on full graph  
        train_acc, train_class_acc = evaluate_embedding_classifier(embeddings, train_y, train_mask, model, num_classes, device)
        valid_acc, valid_class_acc = evaluate_embedding_classifier(embeddings, valid_y, valid_mask, model, num_classes, device)
        if epoch % 10 == 0 or epoch == (NUM_EPOCHS-1):
            print(f"Epoch {epoch} with train loss: {train_loss:.3f} train accuracy: {train_acc:.3f} validation accuracy: {valid_acc:.3f}")
            print("Per class train accuracy: ", train_class_acc)
            print("Per class val accuracy: ", valid_class_acc)
        # store the loss and the accuracy for the final plot
        epoch_stats = {'train_acc': train_acc, 'val_acc': valid_acc, 'epoch':epoch}
        training_stats = update_stats(training_stats, epoch_stats)

    # Lets look at our final test performance
    # Only need to get the node embeddings once, take from the training evaluation call
    test_acc, test_class_acc = evaluate_embedding_classifier(embeddings, test_y, test_mask, model, num_classes, device)
    print(f"Our final test accuracy for the GNN is: {test_acc:.3f}")
    print("Final per class accuracy on test set: ", test_class_acc)

    # Save training stats if on final iteration of the run, the node embeddings are actually passed in for training, where  
    node_embeddings = embeddings
    save_training_info(training_stats, node_embeddings, filename+"_"+str(seed))
    # Save final results
    final_results_list = [seed, test_acc, test_class_acc, train_class_acc, valid_class_acc]
    save_final_results(final_results_list, filename)
    # Save final model weights incase we want to do further inference later
    torch.save(model.state_dict(), file_path+filename+"_" + str(seed) + "_model.pt")
    return training_stats

In [80]:
# Use parameters from example in https://github.com/GTmac/FastRP/blob/master/fast-random-projection-blogcatalog.ipynb
# Except our input matrix in an adjacency matrix and since we are not tuning alpha we just set this to None
input_matrix = 'adj'
alpha = -0.67
conf = {
        'projection_method': 'sparse',
        'input_matrix': input_matrix,
        'weights': [0.0, 0.0, 1.0, 6.67],
        'normalization': True,
        'dim': HIDDEN_DIM,
        'alpha': alpha,
        'C': 0.1
    }

num_nodes = node_features.shape[0]

# Convert adjacency matrix to scipy matrix
adj_matrix = to_scipy_sparse_matrix(edge_indices)
# Whether we are using the adjacency or transition matrix
if input_matrix == 'trans':
  # Create the degree matrix for the graph
  degrees = degree(edge_indices[0])
  diagonal_degree = sp.sparse.spdiags(degrees, 0, degrees.size()[0], degrees.size()[0])
  # Create the transition matrix for the graph = D-1(A)
  transition_matrix = scipy.sparse.linalg.inv(diagonal_degree).multiply(adj_matrix)
  print("Using transition matrix")
else:
  transition_matrix = adj_matrix
  print("Using adjacency matrix")

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
for seed in seeds:
  set_seeds(seed)
  
  embeddings = fastrp_wrapper(transition_matrix, conf, seed)
  # convert to tensor 
  embeddings = torch.from_numpy(embeddings)

  # Create the model
  model = FastRPEmbeddingWrapper(HIDDEN_DIM, num_classes)
  model = model.to(device)

  # Run training loop
  print("TRAINING WITH SEED: ", str(seed))
  train_stats_cora = train_eval_loop_embedding_classifier(model, embeddings, train_y, train_mask, 
                                             valid_y, valid_mask, test_y, test_mask, num_classes, seed, filename+"/"+filename, device, is_cora)
  # Print out graphs if not using GPU
  if device == torch.device('cpu'):
    plot_stats(train_stats_cora, name=filename)

Using adjacency matrix
SETTING SEEDS TO:  4193977854


/usr/local/lib/python3.9/dist-packages/sklearn/preprocessing/_data.py:240: UserWarning: Numerical issues were encountered when centering the data and might not be solved. Dataset may contain too large values. You may need to prescale your features.
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/sklearn/preprocessing/_data.py:259: UserWarning: Numerical issues were encountered when scaling the data and might not be solved. The standard deviation of the data is probably very close to 0. 
  warnings.warn(


TRAINING WITH SEED:  4193977854
Epoch 0 with train loss: 2.867 train accuracy: 21.586 validation accuracy: 21.020
Per class train accuracy:  tensor([0.0778, 0.0830, 0.1024, 0.0856, 0.2584, 0.1186, 0.1441, 0.6841, 0.2860,
        0.0000, 0.0139, 0.1665, 0.6508, 0.2334, 0.5619], device='cuda:0')
Per class val accuracy:  tensor([0.1338, 0.0761, 0.0805, 0.0465, 0.2796, 0.0888, 0.1622, 0.6919, 0.1935,
        0.0417, 0.0035, 0.1376, 0.6429, 0.2382, 0.6400], device='cuda:0')
Epoch 9 with train loss: 1.087 train accuracy: 72.628 validation accuracy: 71.129
Per class train accuracy:  tensor([0.4245, 0.6751, 0.7537, 0.5525, 0.8062, 0.5970, 0.4685, 0.7852, 0.5269,
        0.3714, 0.8811, 0.7998, 0.8214, 0.7465, 0.9181], device='cuda:0')
Per class val accuracy:  tensor([0.4789, 0.5326, 0.7073, 0.4302, 0.7742, 0.5535, 0.4189, 0.7946, 0.4645,
        0.4167, 0.8789, 0.7862, 0.8571, 0.7751, 0.9086], device='cuda:0')
Our final test accuracy for the GNN is: 72.549
Final per class accuracy on test set:

/usr/local/lib/python3.9/dist-packages/sklearn/preprocessing/_data.py:240: UserWarning: Numerical issues were encountered when centering the data and might not be solved. Dataset may contain too large values. You may need to prescale your features.
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/sklearn/preprocessing/_data.py:259: UserWarning: Numerical issues were encountered when scaling the data and might not be solved. The standard deviation of the data is probably very close to 0. 
  warnings.warn(


TRAINING WITH SEED:  1863727779
Epoch 0 with train loss: 2.839 train accuracy: 22.341 validation accuracy: 21.619
Per class train accuracy:  tensor([0.1627, 0.2491, 0.4358, 0.0389, 0.2560, 0.0913, 0.2748, 0.0343, 0.2602,
        0.0143, 0.1663, 0.3060, 0.0119, 0.1350, 0.7257], device='cuda:0')
Per class val accuracy:  tensor([0.2254, 0.2609, 0.4220, 0.0116, 0.2545, 0.0752, 0.2027, 0.0162, 0.2194,
        0.0833, 0.1661, 0.3194, 0.0000, 0.1427, 0.6229], device='cuda:0')
Epoch 9 with train loss: 1.054 train accuracy: 73.137 validation accuracy: 71.238
Per class train accuracy:  tensor([0.4693, 0.6029, 0.7203, 0.4903, 0.7943, 0.6388, 0.6171, 0.7798, 0.5419,
        0.5429, 0.8718, 0.7801, 0.8135, 0.7694, 0.9124], device='cuda:0')
Per class val accuracy:  tensor([0.4577, 0.5000, 0.6878, 0.3721, 0.7706, 0.5763, 0.5811, 0.8054, 0.4774,
        0.5833, 0.8616, 0.7813, 0.8452, 0.7799, 0.8971], device='cuda:0')
Our final test accuracy for the GNN is: 73.039
Final per class accuracy on test set:

...

/usr/local/lib/python3.9/dist-packages/sklearn/preprocessing/_data.py:240: UserWarning: Numerical issues were encountered when centering the data and might not be solved. Dataset may contain too large values. You may need to prescale your features.
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/sklearn/preprocessing/_data.py:259: UserWarning: Numerical issues were encountered when scaling the data and might not be solved. The standard deviation of the data is probably very close to 0. 
  warnings.warn(


TRAINING WITH SEED:  3303475786
Epoch 0 with train loss: 2.883 train accuracy: 21.432 validation accuracy: 20.447
Per class train accuracy:  tensor([0.1415, 0.0469, 0.1976, 0.1440, 0.3170, 0.2662, 0.0946, 0.1047, 0.0344,
        0.2714, 0.4815, 0.2330, 0.0000, 0.1217, 0.5162], device='cuda:0')
Per class val accuracy:  tensor([0.1479, 0.0652, 0.1732, 0.1163, 0.2688, 0.2574, 0.0405, 0.1351, 0.0387,
        0.1250, 0.5225, 0.2211, 0.0000, 0.1125, 0.4743], device='cuda:0')
Epoch 9 with train loss: 1.063 train accuracy: 73.510 validation accuracy: 71.565
Per class train accuracy:  tensor([0.4292, 0.6498, 0.8000, 0.5486, 0.7656, 0.6646, 0.3874, 0.7780, 0.6151,
        0.5571, 0.8845, 0.7974, 0.7976, 0.7328, 0.9162], device='cuda:0')
Per class val accuracy:  tensor([0.4789, 0.5543, 0.7659, 0.5233, 0.7384, 0.6173, 0.3649, 0.7784, 0.5161,
        0.2500, 0.8962, 0.7838, 0.8571, 0.7304, 0.9086], device='cuda:0')
Our final test accuracy for the GNN is: 72.958
Final per class accuracy on test set:

/usr/local/lib/python3.9/dist-packages/sklearn/preprocessing/_data.py:240: UserWarning: Numerical issues were encountered when centering the data and might not be solved. Dataset may contain too large values. You may need to prescale your features.
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/sklearn/preprocessing/_data.py:259: UserWarning: Numerical issues were encountered when scaling the data and might not be solved. The standard deviation of the data is probably very close to 0. 
  warnings.warn(


TRAINING WITH SEED:  2952735006
Epoch 0 with train loss: 2.938 train accuracy: 18.130 validation accuracy: 17.121
Per class train accuracy:  tensor([0.0731, 0.5415, 0.0634, 0.1751, 0.2667, 0.1970, 0.1622, 0.0686, 0.2624,
        0.0286, 0.1224, 0.0861, 0.6667, 0.1024, 0.7162], device='cuda:0')
Per class val accuracy:  tensor([0.0845, 0.4239, 0.0659, 0.1744, 0.2760, 0.1526, 0.1622, 0.0865, 0.2194,
        0.0000, 0.1176, 0.1007, 0.7381, 0.0859, 0.6914], device='cuda:0')
Epoch 9 with train loss: 1.133 train accuracy: 71.546 validation accuracy: 70.256
Per class train accuracy:  tensor([0.4080, 0.6282, 0.6976, 0.6187, 0.8134, 0.6829, 0.4144, 0.7834, 0.5871,
        0.5571, 0.8695, 0.7646, 0.8333, 0.6888, 0.9162], device='cuda:0')
Per class val accuracy:  tensor([0.4718, 0.5217, 0.7073, 0.5116, 0.7563, 0.6446, 0.3649, 0.8054, 0.4903,
        0.5000, 0.8685, 0.7715, 0.8690, 0.6941, 0.9029], device='cuda:0')
Our final test accuracy for the GNN is: 72.440
Final per class accuracy on test set:

/usr/local/lib/python3.9/dist-packages/sklearn/preprocessing/_data.py:240: UserWarning: Numerical issues were encountered when centering the data and might not be solved. Dataset may contain too large values. You may need to prescale your features.
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/sklearn/preprocessing/_data.py:259: UserWarning: Numerical issues were encountered when scaling the data and might not be solved. The standard deviation of the data is probably very close to 0. 
  warnings.warn(


# Node2Vec

In [ ]:
from torch_geometric.nn import Node2Vec
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

data_name = "Arxiv"

# Get masks and training labels for each split
if data_name == "Cora":
  num_classes = 7
  data = cora_data
  # Get the edge indices and node features for our model
  edge_indices = data.edge_index
  node_features = data.x
  # CHANGE: To name of model being tested
  filename =  "Node2Vec_Cora"
  train_mask = data.train_mask
  train_y = data.y[train_mask]
  valid_mask = data.val_mask
  valid_y = data.y[valid_mask]
  test_mask = data.test_mask
  test_y = data.y[test_mask]
elif data_name == "Coauthor":
  data = cs_data
  # Get the edge indices and node features for our model
  edge_indices = data.edge_index
  node_features = data.x
  num_classes = 15
  filename =  "Node2Vec_Coauthor_CS"
  train_mask = train_mask_cs
  train_y = data.y[train_mask]
  valid_mask = val_mask_cs
  valid_y = data.y[valid_mask]
  test_mask = test_mask_cs
  test_y = data.y[test_mask]
elif data_name == "Arxiv":
  data = arxiv_data
  edge_indices = arxiv_data.edge_index
  node_features = arxiv_data.x
  neighbour_dataset = arxiv_data

  # Get masks and training labels for each split
  train_mask = train_idx
  train_y = arxiv_data.y[train_mask]
  valid_mask = valid_idx
  valid_y = arxiv_data.y[valid_mask]
  test_mask = test_idx
  test_y = arxiv_data.y[test_mask]

  num_classes = 40
  is_cora = False

device = 'cuda' if torch.cuda.is_available() else 'cpu'

# use 30 seeds which have been randomly generated using seed_list = [np.random.randint(4294967296 - 1) for i in range(30)]
seeds = [4193977854, 1863727779, 170173784, 2342954646, 116846604, 2105922959, 2739899259, 1024258131, 806299656, 880019963, 1818027900, 2135956485, 3710910636, 1517964140, 4083009686, 2455059856, 400225693, 89475662, 361232447, 3647665043, 1221215631, 2036056847, 1860537279, 516507873, 3692371949, 3300171104, 2794978777, 3303475786, 2952735006, 572297925]

# create folder for saving all model info into if it does not exist already
if not os.path.exists(file_path+filename+"/"):
  os.mkdir(file_path+filename+"/")

filename = filename + "/" + filename

for seed in seeds:
  set_seeds(seed)
  # Create the model
  #model = GATModelWrapper(in_channels = node_features.shape[-1], hidden_channels = node_features.shape[-1], num_layers=1, out_channels=num_classes, v2=True)
  model = Node2VecWrapper(data.edge_index.to(device), embedding_size=128, walk_length=20,
                     context_size=10, walks_per_node=10,
                     num_negative_samples=1, p=1, q=1, sparse=True, out_channels=num_classes).to(device)
  loader = model.loader(batch_size=128, shuffle=True,
                      num_workers=0)
  optimizer = torch.optim.SparseAdam(list(model.parameters()), lr=0.01)

  def train():
    model.train()
    total_loss = 0
    for pos_rw, neg_rw in loader:
      optimizer.zero_grad()
      loss = model.loss(pos_rw.to(device), neg_rw.to(device))
      loss.backward()
      optimizer.step()
      total_loss += loss.item()
    return total_loss / len(loader)

  @torch.no_grad()
  def find_model_acc(model, train_z, train_y, test_z, test_y, solver: str = 'lbfgs', multi_class: str = 'auto', *args, **kwargs):
    pred_y = model.test(train_z, train_y, test_z, test_y, solver=solver, multi_class=multi_class, *args, **kwargs)
    acc = accuracy_score(test_y.detach().cpu().numpy(), pred_y)
    matrix = confusion_matrix(test_y.detach().cpu().numpy(), pred_y)
    per_class_acc = matrix.diagonal()/matrix.sum(axis=1)
    #print(m)
    #report = classification_report(test_y.detach().cpu().numpy(), pred_y)
    #print(report)
    return acc, per_class_acc

  @torch.no_grad()
  def test():
    model.eval()
    
    pred, z = model()
    acc_train, per_class_train_acc = find_model_acc(model, z[train_mask], data.y[train_mask],
                      z[train_mask], data.y[train_mask])
  
    acc_val, per_class_val_acc = find_model_acc(model, z[train_mask], data.y[train_mask],
                      z[valid_mask], data.y[valid_mask])

    acc_test, per_class_test_acc = find_model_acc(model, z[train_mask], data.y[train_mask],
                      z[test_mask], data.y[test_mask])

    return z, acc_train, per_class_train_acc, acc_val, per_class_val_acc, acc_test, per_class_test_acc

  training_stats = None
  for epoch in range(0, 10):
    loss = train()
    node_embeddings, acc_train, per_class_train_acc, acc_val, per_class_val_acc, acc_test, per_class_test_acc = test()
    print(f'Epoch: {epoch:02d}, Loss: {loss:.4f}, Acc_train: {acc_train:.4f}, Acc_val: {acc_val:.4f}, Acc_test: {acc_test:.4f}')
    print(f'Per class train accuracy: ', per_class_train_acc)
    epoch_stats = {'train_acc': acc_train, 'val_acc': acc_val, 'test_acc': acc_test, 'epoch':epoch}
    training_stats = update_stats(training_stats, epoch_stats)
  
  # Save training stats if on final iteration of the run
  save_training_info(training_stats, node_embeddings, filename+"_"+str(seed))
  # Save final results
  final_results_list = [seed, acc_test, per_class_test_acc, per_class_train_acc, per_class_val_acc]
  save_final_results(final_results_list, filename)
  # Save final model weights incase we want to do further inference later
  torch.save(model.state_dict(), file_path+filename+"_" + str(seed) + "_model.pt")

  plot_stats(training_stats, name=filename)

# Similarity tests

https://github.com/SGDE2020/embedding_stability/blob/master/similarity_tests/similarity_tests.py